In [1]:
from datascience import *
import numpy as np
from numpy import random
import pandas as pd
from math import *
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

## Lesson 32: Likelihood Ratio Tests

Last time, we introduced Likelihood Ratio tests. Recall that the point of a likelihood ratio test is to compare the likelihood function under a hypothesized value of the parameter with the liklihood function at its maximum. Instead of looking at the ratio $\Lambda$ itself, we often consider $-2\log \Lambda$ instead, since it has a handy distribution. 

### Example 1: Exponential Distribution

Suppose $X_1,X_2,...,X_n$ is an iid sequence of random variables from the exponential distribution with unknown parameter $\lambda$. Recall that the maximum likelihood estimate of $\lambda$ is $1\over\bar{X}$. We collect a random sample of size 20 and want to test the hypothesis $H_0: \lambda = 3$ vs $H_1: \lambda \neq 3$. Using the data in the python box below, conduct a likelihood ratio test on this hypothesis.  

In [2]:
my_data=np.array([0.18,0.277,0.105,0.126,0.225,0.026,0.123,0.423,0.006,0.281,0.050,0.692,0.105,0.275,0.346,0.079,0.045,0.222,0.063,0.281])

In [3]:
xbar = np.mean(my_data)
print("Our maximum likelihood estimate of lambda for this draw is", 1/xbar)

Our maximum likelihood estimate of lambda for this draw is 5.089058524173028


In [4]:
lambda_ML = 1/xbar
print("Lambda is", lambda_ML)
likelihood_ML = np.prod(lambda_ML * e**(-lambda_ML * my_data))
print("Max Liklihood of the data is", likelihood_ML)
null_likelihood = np.prod(3 * e**(-3*my_data))
print("Under null hypothesis, max likelihood is", null_likelihood, ", but how weird is this?")

Lambda is 5.089058524173028
Max Liklihood of the data is 279807.44620464457
Under null hypothesis, max likelihood is 26429.755940265666 , but how weird is this?


In [5]:
capital_lambda = null_likelihood / likelihood_ML
log_lambda = -2*log(capital_lambda)
p_value = stats.chi2.sf(log_lambda, 1)
print("p-value:", p_value)

p-value: 0.02982722919477517


With a p-value of 0.0298, we can conclude that it is very unlikely that lambda would be equal to 3.

#### Power

Suppose that the true value of $\lambda$ is 5. Let's determine the power of this test. Let $n=20$. Then determine the power if $n=50$. Remember, power is the probability of correctly rejecting the null hypothesis. 

First, find what value of $-2 \log \Lambda$ would lead you to reject $H_0$. This is sometimes called the critical value. 

In [6]:
crit=stats.chi2.ppf(0.95, 1)
crit

3.841458820694124

*...So 3.84 is the threshold of weirdness. Anything above this we reject, anything below we fail to reject and say it could have viably happened by chance.*

Next, obtain the power. Obtain a sample of size 20 from the true population and obtain the value of $-2\log \Lambda$ for this sample. Repeat many times and determine how often you reject the null hypothesis. 

In [7]:
values = np.array([])
n = 20

for _ in np.arange(10000):
    data = stats.expon.rvs(scale=1/5, size=n)
    lam = 1 / np.mean(data)
    null_lh = np.prod(3 * e**(-3*data))
    max_lh = np.prod(lam * e**(-lam*data))
    cap_lam = null_lh / max_lh
    log_lam = -2*log(cap_lam)
    values = np.append(values, log_lam)
    
np.mean(values>=crit)    

0.5946

Repeat for a sample size of 50. What do you expect to happen to power? 

In [8]:
# I expect power to increase
lamvals = np.array([])
n = 50
lam_0 = 3

for _ in np.arange(10000):
    sample_data = stats.expon.rvs(scale=1/5, size=n)
    lam_new = 1/np.mean(sample_data)
    null_lh = np.prod(lam_0 * e**(-lam_0*sample_data))
    max_lh = np.prod(lam_new * e**(-lam_new*sample_data))
    cap_lam = null_lh / max_lh
    log_lam = -2*log(cap_lam)
    lamvals = np.append(lamvals, log_lam)
    
p_val = np.mean(lamvals >= crit)
print("p-value", p_val)

p-value 0.9524


### A Different Test

We've explored hypothesis tests in this class before. Taking advantage of our computing power, we don't have to rely on test statistics with asymptotic distributions. Let's conduct a more direct hypothesis test using simulation. Recall:

$$
H_0: \lambda = 3
$$

$$
H_1: \lambda \neq 3
$$

Pick a different test statistic. Obtain an empirical distribution of that test statistic under $H_0$. Next, find the $p$-value by determining how often this test statistic is at or further away from the test statistic derived from the sample. Remember that this is a two-sided test. 

In [20]:
test_stats = np.array([])
n = 1000
lam_0 = 3

# Test stat will be difference between lam_new and lam_0
for _ in np.arange(10000):
    sample_data = stats.expon.rvs(scale=1/5, size=n)
    lam_new = 1/np.mean(sample_data)
    test_stat = lam_new - lam_0
    test_stats = np.append(test_stats, test_stat)

print(percentile([2.5, 97.5], test_stats))
print("p-value", np.mean(test_stats<1.7097) + np.mean(test_stats>2.3196))

[1.70477062 2.32382541]
p-value 0.054


How did the $p$-value compare to the LRT $p$-value? I wonder how the power of this test compares to our LRT. 

This p-value is much smaller, but still small enough that we reject the null hypothesis.

#### Power

Let's figure out the power of this test. First, determine for what values of the test statistic would lead us to reject $H_0$. These values can be referred to as your rejection region. 

In [16]:
percentile([2.5, 97.5], test_stats)

array([0.86738375, 3.71917905])

Now, determine the power of this test. Like in the LRT case, obtain a sample of size 20 and obtain the test statistic. Repeat many times and see how often your test statistic is in your rejection region. 

In [21]:
test_stats = np.array([])
n = 20
lam_0 = 3

# Test stat will be difference between lam_new and lam_0
for _ in np.arange(10000):
    sample_data = stats.expon.rvs(scale=1/5, size=n)
    lam_new = 1/np.mean(sample_data)
    test_stat = lam_new - lam_0
    test_stats = np.append(test_stats, test_stat)

print(percentile([2.5, 97.5], test_stats))
print("p-value", np.mean(test_stats<0.34456) + np.mean(test_stats>5.1299))

[0.38663494 5.1689812 ]
p-value 0.0463


Repeat for a sample size of 50. Note that you will have to obtain new critical values in order to do this.  

In [22]:
test_stats = np.array([])
n = 50
lam_0 = 3

# Test stat will be difference between lam_new and lam_0
for _ in np.arange(10000):
    sample_data = stats.expon.rvs(scale=1/5, size=n)
    lam_new = 1/np.mean(sample_data)
    test_stat = lam_new - lam_0
    test_stats = np.append(test_stats, test_stat)

print(percentile([2.5, 97.5], test_stats))
print("p-value", np.mean(test_stats>3.7788) + np.mean(test_stats<0.8793))

[0.86674933 3.79348595]
p-value 0.0523
